In [50]:
import findspark
findspark.init('/home/nick/spark-3.0.1-bin-hadoop2.7')

from pyspark.sql import SparkSession
from pyspark.sql.functions import length
from pyspark.ml.feature import Tokenizer, StopWordsRemover, CountVectorizer, IDF, StringIndexer, VectorAssembler
from pyspark.ml.classification import NaiveBayes
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import MulticlassClassificationEvaluator


spark = SparkSession.builder.appName('Spam Detection').getOrCreate()

In [24]:
data = spark.read.csv('Natural_Language_Processing/smsspamcollection/SMSSpamCollection',
                     inferSchema=True, sep='\t')
data.show(5)
data = data.withColumnRenamed('_c0','class').withColumnRenamed('_c1','text')
data.show(5)

+----+--------------------+
| _c0|                 _c1|
+----+--------------------+
| ham|Go until jurong p...|
| ham|Ok lar... Joking ...|
|spam|Free entry in 2 a...|
| ham|U dun say so earl...|
| ham|Nah I don't think...|
+----+--------------------+
only showing top 5 rows

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
+-----+--------------------+
only showing top 5 rows



In [25]:
data = data.withColumn('length',length(data['text']))
data.show(5)

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
+-----+--------------------+------+
only showing top 5 rows



In [26]:
data.groupBy('class').mean().show()

+-----+-----------------+
|class|      avg(length)|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



In [29]:
tokenizer = Tokenizer(inputCol='text', outputCol='token_text')
stop_remove = StopWordsRemover(inputCol='token_text', outputCol='stop_token')
count_vec = CountVectorizer(inputCol='stop_token', outputCol='c_vec')
idf = IDF(inputCol='c_vec', outputCol='tf_idf')
ham_spam_to_numeric = StringIndexer(inputCol='class', outputCol='label')

clean_up = VectorAssembler(inputCols=['tf_idf', 'length'], outputCol='features')

## Frequently for NLP use a Naive Bayes for classification

In [32]:
nb = NaiveBayes()

In [35]:
pipe_dataprep = Pipeline(stages=[ham_spam_to_numeric,
                                tokenizer,
                                stop_remove,
                                count_vec,
                                idf,
                                clean_up])

In [36]:
cleaner = pipe_dataprep.fit(data)

In [39]:
clean_data = cleaner.transform(data)
clean_data.columns

['class',
 'text',
 'length',
 'label',
 'token_text',
 'stop_token',
 'c_vec',
 'tf_idf',
 'features']

In [43]:
clean_data = clean_data.select(['label','features'])
clean_data.show(5)

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(13424,[7,11,31,6...|
|  0.0|(13424,[0,24,297,...|
|  1.0|(13424,[2,13,19,3...|
|  0.0|(13424,[0,70,80,1...|
|  0.0|(13424,[36,134,31...|
+-----+--------------------+
only showing top 5 rows



In [44]:
train_data, test_data = clean_data.randomSplit([0.7,0.3])

In [46]:
spam_detector = nb.fit(train_data)

In [47]:
data.printSchema()

root
 |-- class: string (nullable = true)
 |-- text: string (nullable = true)
 |-- length: integer (nullable = true)



In [49]:
test_results = spam_detector.transform(test_data)
test_results.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(13424,[0,1,2,13,...|[-601.98424404768...|[1.0,6.6072904158...|       0.0|
|  0.0|(13424,[0,1,2,41,...|[-1061.5876846148...|[1.0,9.2516301993...|       0.0|
|  0.0|(13424,[0,1,7,15,...|[-654.87738773247...|[1.0,5.5276227570...|       0.0|
|  0.0|(13424,[0,1,9,14,...|[-536.18355575630...|[1.0,6.6123033884...|       0.0|
|  0.0|(13424,[0,1,11,32...|[-872.69658898342...|[1.0,1.0702129072...|       0.0|
|  0.0|(13424,[0,1,12,33...|[-444.93497434011...|[1.0,4.8381063288...|       0.0|
|  0.0|(13424,[0,1,14,31...|[-216.97987788817...|[1.0,2.2729672683...|       0.0|
|  0.0|(13424,[0,1,31,43...|[-337.93079674438...|[1.0,5.2300980274...|       0.0|
|  0.0|(13424,[0,1,43,69...|[-615.11841881311...|[0.99996222296317...|       0.0|
|  0.0|(13424,[0

In [53]:
accuracy_eval = MulticlassClassificationEvaluator()
accuracy = accuracy_eval.evaluate(test_results)
print(accuracy)

0.9176498128548239
